In [6]:
import os
import sys

if 'has_changed_dir' not in globals():
    repo_path = os.path.abspath(os.path.join('..'))
    
    if repo_path not in sys.path:
        sys.path.append(repo_path)
    
    os.chdir(repo_path)
    
    globals()['has_changed_dir'] = True
    print(repo_path)
    print(os.getcwd())


# from typing import Optional, Union

# import torch
from transformers import AutoModelForSeq2SeqLM, NllbTokenizer

from src.train_model import LightningModel




### Load model using ckpt (you dont need it if you convert your model to transformers)

In [7]:
# model_path: str = "models/re-init/"
# tokenizer_path: str = "models/re-init/"
# tokenizer_vocab_file_path: str = "models/re-init/sentencepiece.bpe.model"
# ckpt_path: Optional[Union[str, None]] = "models/it_1/epoch=17-val_loss=1.48184.ckpt"

# model = AutoModelForSeq2SeqLM.from_pretrained(model_path, local_files_only=True)
# model.eval()
# tokenizer = NllbTokenizer.from_pretrained(tokenizer_path, vocab_file = tokenizer_vocab_file_path)


# # Option 1:
# # lightning_model = LightningModel.load_from_checkpoint(ckpt_path, map_location=torch.device("cuda:1"), model = model, tokenizer = tokenizer)

# # Option 2:
# ckpt = torch.load(ckpt_path, map_location=torch.device("cuda:1"))
# lightning_model = LightningModel(model, tokenizer)
# lightning_model.load_state_dict(ckpt['state_dict'])


### Load converted model

In [8]:
""" Load local converted model"""

# final_model_path = "models/final"

# model = AutoModelForSeq2SeqLM.from_pretrained(final_model_path)
# tokenizer = NllbTokenizer.from_pretrained(final_model_path, vocab_file = final_model_path + "/sentencepiece.bpe.model")
# lightning_model = LightningModel(model, tokenizer)

""" Download and load converted model from hub """

model = AutoModelForSeq2SeqLM.from_pretrained("leks-forever/nllb-200-distilled-600M")
tokenizer = NllbTokenizer.from_pretrained("leks-forever/nllb-200-distilled-600M")
lightning_model = LightningModel(model, tokenizer)

In [9]:
sentence: str = "Римдин аскерар ва гьакӀни чӀехи хахамрини  фарисейри ракъурнавай нуькерар Ягьуд галаз багъдиз атана. Абурув виридав яракьар, чирагъар ва шемгьалар гвай."

translation = lightning_model.predict(sentence, src_lang='lez_Cyrl', tgt_lang='rus_Cyrl')

print(translation)

['Иуда вместе с римским войском и другими посланными к нему царями священнослужителями пришли в сад. Они были вооружёны, в руках лампады и лампады.']


In [10]:
sentence: str = "Когда Исо услышал это, Он сказал: – Не здоровым нужен врач, а больным."

translation = lightning_model.predict(sentence, src_lang='rus_Cyrl' , tgt_lang='lez_Cyrl')

print(translation)

# Реальный перевод: И гафар ван хьайила, Исади абуруз лагьана: – Жерягь сагъбуруз ваъ, начагъбуруз герек я.

['И гафарин ван хьайила, Исади абуруз лагьана: - Жерягь сагъбуруз ваъ, начагъбуруз герек я.']


In [11]:


sentences = [
    "Давай поедим пиццу, а потом еще немного погуляем!",
    "Я люблю гулять по парку ранним утром, когда воздух свежий и тишина вокруг.",
    "Новый фильм, который мы смотрели вчера, произвёл на меня сильное впечатление",
    "В следующем году я планирую посетить несколько стран, чтобы познакомиться с их культурой.",
    "После долгого рабочего дня приятно расслабиться с книгой и чашкой чая."
]

for sentence in sentences:
    translation = lightning_model.predict(sentence, src_lang='rus_Cyrl' , tgt_lang='lez_Cyrl')

    print(translation)


['Ша чна са тӀуьнни тӀуьна, ахпа мад са тӀимил къекъуьн патал чара жен!']
['Заз пакамахъ, хъсан гар алаз, сагъ-саламатдиз къекъвез кӀанзава.']
['Чна йифиз килигай цӀийиди заз гзаф таъсир авуна.']
['Зун къведай йисуз са шумуд уьлкведиз, абурун адетралди чир хьун патал, къвез кӀанзава.']
['Гзаф кӀвалах авурдалай кьулухъ ктаб ва са гъам гваз ял акьадайвал хъсан я.']


In [12]:
sentences = [

]

for sentence in sentences:
    translation = lightning_model.predict(sentence, src_lang='lez_Cyrl' , tgt_lang='rus_Cyrl')

    print(translation)


### Note: 
You can use the predict method regardless of the Lightning model

In [13]:
def predict(text, src_lang='lez_Cyrl', tgt_lang='rus_Cyrl', a=32, b=3, max_input_length=1024, num_beams=1, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)


In [14]:
sentence: str = "Римдин аскерар ва гьакӀни чӀехи хахамрини  фарисейри ракъурнавай нуькерар Ягьуд галаз багъдиз атана. Абурув виридав яракьар, чирагъар ва шемгьалар гвай."

translation = predict(sentence, src_lang='lez_Cyrl', tgt_lang='rus_Cyrl')

print(translation)

['Иуда вместе с римским войском и другими посланными к нему царями священнослужителями пришли в сад. Они были вооружёны, в руках лампады и лампады.']
